In [2]:
import pandas as pd
import numpy as np
import holidays

INPUT_PATH = "C:/Users/u00378/Desktop/tcc_est_unb"
#INPUT_PATH = "C:/Users/Igor/Desktop/TCC"

df = pd.read_pickle(f'{INPUT_PATH}/dados/tcc_data.pkl')

In [12]:
teste = df[list(map(lambda x: True if x.year == 2007 else False, df['data_inversa']))]

In [23]:
teste.sample(frac=0.0001, random_state=1)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
3344,178489.0,2007-01-11,Quinta,11:00:00,MG,381,421,SABARA,Outras,Colisão lateral,...,0,3,0,0,3,NaN,NaN,NaN,NaN,NaN
73910,265086.0,2007-08-11,Sábado,20:15:00,RN,101,96,NATAL,Falta de atenção,Colisão lateral,...,0,2,0,0,2,NaN,NaN,NaN,NaN,NaN
95132,289199.0,2007-10-12,Sexta,05:55:00,MG,135,624.6,CURVELO,Falta de atenção,Colisão Transversal,...,0,1,0,1,2,NaN,NaN,NaN,NaN,NaN
15340,193456.0,2007-02-17,Sábado,18:55:00,RR,401,3.9,CANTA,Outras,Colisão lateral,...,1,1,0,1,2,NaN,NaN,NaN,NaN,NaN
16321,194575.0,2007-02-20,Terça,15:00:00,PA,316,23,BENEVIDES,Outras,Saída de Pista,...,0,1,0,1,1,NaN,NaN,NaN,NaN,NaN
31191,212953.0,2007-04-08,Domingo,16:40:00,BA,324,516.8,FEIRA DE SANTANA,Falta de atenção,Colisão traseira,...,0,3,0,0,3,NaN,NaN,NaN,NaN,NaN
17381,195856.0,2007-02-19,Segunda,16:10:00,BA,101,45,ESPLANADA,Outras,Saída de Pista,...,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN
101816,296674.0,2007-10-27,Sábado,17:00:00,DF,20,15.9,BRASILIA,Ingestão de álcool,Colisão Transversal,...,0,2,0,0,2,NaN,NaN,NaN,NaN,NaN
62695,251529.0,2007-07-10,Terça,06:50:00,MG,381,249,IPATINGA,Falta de atenção,Colisão traseira,...,1,1,0,1,2,NaN,NaN,NaN,NaN,NaN
45485,230492.0,2007-05-18,Sexta,07:35:00,RS,386,297.6,POUSO NOVO,Falta de atenção,Colisão traseira,...,0,2,0,0,2,NaN,NaN,NaN,NaN,NaN


## WEEKDAY AND HOLIDAYS

In [3]:
br_holidays = holidays.country_holidays('BR')  # this is a dict
df_holidays = holidays.country_holidays('BR', subdiv='DF')  # this is a dict


In [4]:
BR_H = list(map(lambda x: br_holidays.get(x), df['data_inversa']))
DF_H = list(map(lambda x: df_holidays.get(x), df['data_inversa']))

WEEKDAYS = list(map(lambda x: x.weekday(), df['data_inversa']))

#df['event_name'] = map(br_holidays.get(), df['data_inversa'])

In [5]:
BR_H == DF_H

False